In [1]:
import matplotlib
import numpy as np
import math
import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import random
import scipy as sp
import torch
from torch.nn import Linear
from torch import Tensor
from torch.nn import MSELoss
from torch.optim import SGD, Adam, RMSprop
from torch.autograd import Variable, grad
from torch.utils.data.sampler import SubsetRandomSampler,WeightedRandomSampler
from sklearn.metrics import r2_score
from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing

import sys
import pandas as pd

manualSeed = 1 # random.randint(1, 10000) # Set the desired seed to reproduce the results
torch.manual_seed(manualSeed)

np.random.seed(1)

In [2]:
def load_checkpoint(filepath,device):
    '''
        Loading an instance of EikoNet from a saved .pt
    
    
        INPUTS:
            filepath - path to .pt file
            
        OUTPUTS:
            model - Eikonet Model
    '''
    
    
    checkpoint = torch.load(filepath)
    # Defining Neural Network
    net = Reg_model().to(device)
    net.apply(init_weights)
    net.float()
    net.to(device)
    optimizer  = torch.optim.Adam(net.parameters())
    scheduler = []
    model = Model(net, optimizer,scheduler, model_path=filepath,model_name='')
    model.total_train_loss = checkpoint['train_loss']
    model.total_val_loss   = checkpoint['val_loss']

    model.network.load_state_dict(checkpoint['model_state_dict'])
    for parameter in model.network.parameters():
        parameter.requires_grad = False

    model.network.eval()
    return model

In [3]:
class Reg_model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the neural network
        self.relu = torch.nn.ReLU()
        m = 1
        # Layers 
        self.fc0  = Linear(299,299*m)
        self.fc1  = Linear(299*m,299*m)

        # resnet - block 1
        self.rn1_fc1  = Linear(299*m,299*m)
        self.rn1_fc2  = Linear(299*m,299*m)
        self.rn1_fc3  = Linear(299*m,299*m)

        # resnet - block 2 
        self.rn2_fc1  = Linear(299*m,299*m)
        self.rn2_fc2  = Linear(299*m,299*m)
        self.rn2_fc3  = Linear(299*m,299*m)

        # resnet - block 3
        self.rn3_fc1  = Linear(299*m,299*m)
        self.rn3_fc2  = Linear(299*m,299*m)
        self.rn3_fc3  = Linear(299*m,299*m)


        # Output structure
        self.fc8  = Linear(299*m,299)
        self.fc9  = Linear(299,125)

    def forward(self, x):
        
        x   = self.relu(self.fc0(x))
        x   = self.relu(self.fc1(x))

        # Resnet - Block 1
        x0  = x
        x   = self.relu(self.rn1_fc1(x))
        x   = self.relu(self.rn1_fc3(x) + self.rn1_fc2(x0))


        # Resnet - Block 2
        x0  = x
        x   = self.relu(self.rn2_fc1(x))
        x   = self.relu(self.rn2_fc3(x)+self.rn2_fc2(x0))

#         # Resnet - Block 3
        x0  = x
        x   = self.relu(self.rn3_fc1(x))
        x   = self.relu(self.rn3_fc3(x)+self.rn3_fc2(x0))



        # Joining two blocks
        x     = self.relu(self.fc8(x))
        x   = self.fc9(x)
        return x

In [4]:
def init_weights(m):
    if type(m) == torch.nn.Linear:
        stdv = (1. / math.sqrt(m.weight.size(1))/1.)*2
        m.weight.data.uniform_(-stdv,stdv)
        m.bias.data.uniform_(-stdv,stdv)

In [5]:
class Model():
    def __init__(self, network, optimizer,scheduler, model_path, model_name):
        self.network    = network
        self.optimizer  = optimizer
        self.model_path = model_path
        self.model_name = model_name
        self.scheduler  = scheduler

        self.total_train_loss = []
        self.total_val_loss   = []
        
    def train(self, dataset,n_epochs):
        from torch.autograd import Variable
        import time

        len_dataset         = len(dataset)
        n_batches           = int(len(dataset)/btsize + 1)
        training_start_time = time.time()

        # Sending the data to CPU
        dataset.send_device(device) 

        # --------- Splitting the dataset into training and validation -------
        indices            = list(range(int(len_dataset)))
        validation_idx     = indices[:int(len_dataset*(validation_per/100))]  #np.random.choice(indices, size=int(len_dataset*(validation_per/100)), replace=False)
        train_idx          = list(set(indices) - set(validation_idx))
        validation_sampler = SubsetRandomSampler(validation_idx)
        train_sampler      = SubsetRandomSampler(train_idx)

        train_loader       = torch.utils.data.DataLoader(
            dataset,
            batch_size=btsize,
            sampler=train_sampler,
            )    
        validation_loader  = torch.utils.data.DataLoader(
            dataset,
            batch_size=btsize,
            sampler=validation_sampler,
        )    


        for epoch in range(n_epochs):
            print_every           = 1
            start_time            = time.time()
            running_sample_count  = 0

            total_train_loss      = 0
            total_val_loss        = 0


            for i, data in enumerate(train_loader, 0):

                inputs, labels, indexbatch = data
                # Making sure input and labels are floats
                inputs.float()
                labels.float()
                
                inputs.requires_grad_()

                # Forward pass
                pred_labels = self.network(inputs)

                loss_ = EikonalLoss(labels,pred_labels)

                # Update parameters
                loss_.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()


            # ----- Determining the Training Loss -----
            for i, data_train in enumerate(train_loader, 0):
                inputs_train, labels_train, indexbatch_train = data_train
                inputs_train.requires_grad_()
#                 outputs_train,pred_labels_train = self.network(inputs_train)
                pred_labels_train = self.network(inputs_train)
                train_loss              = EikonalLoss(labels_train,pred_labels_train)
                total_train_loss           += train_loss.item()


            # ----- Determining the Validation Loss -----
            for i, data_val in enumerate(validation_loader, 0):
                inputs_val, labels_val, indexbatch_val = data_val
                inputs_val.requires_grad_()
                pred_labels_val = self.network(inputs_val)
#                 outputs_val,pred_labels_val = self.network(inputs_val)
                val_loss                 = EikonalLoss(labels_val,pred_labels_val)
                total_val_loss             += val_loss.item()
#                 del inputs_val, labels_val, indexbatch_val, pred_labels_val, val_loss, wv




            # Creating a running loss for both training and validation data
            total_val_loss   /= len(validation_loader)
            total_train_loss /= len(train_loader)
            self.total_train_loss.append(total_train_loss)
            self.total_val_loss.append(total_val_loss)
            self.scheduler.step(total_val_loss)

#             del train_loader_wei,train_sampler_wei

            if (epoch+1) % 10 == 1:
                with torch.no_grad():
                    print("Epoch = {} -- Training loss = {:.4e} -- Validation loss = {:.4e}".format(epoch+1,total_train_loss,total_val_loss))

            if (epoch+1) % 50 == 1:
                with torch.no_grad():
                    torch.save({
                        'epoch':k,
                        'model_state_dict': self.network.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'train_loss': self.total_train_loss,
                        'val_loss': self.total_val_loss,
                        }, '{}/{}_{}_{}.pt'.format(self.model_path, self.model_name,str(epoch).zfill(5),total_val_loss))

In [6]:
# -------------------------------------------------------------------
# device             = torch.device("cuda:{}".format(GPUid))
device = torch.device("cpu")
torch.set_num_threads(1)

# --------- Defining the neural network -------
# torch.cuda.set_device(device)
net = Reg_model()
net.apply(init_weights)
net.float()
net.to(device)

def count_parameters(net):
    return sum(p.numel() for p in net.parameters() if p.requires_grad)
print(count_parameters(net))

1113900


In [7]:
start = 596
step = 2
border = 895

Xp_1 = np.load('Data/Xp.npy')
Xp_2 = np.load('Data/Xp_1.npy')
Xp_initial = np.concatenate((Xp_1,Xp_2))

Yp_1 = np.load('Data/Yp.npy')
Yp_2 = np.load('Data/Yp_1.npy')
Yp_initial = np.concatenate((Yp_1,Yp_2))


Data = np.column_stack((Xp_initial,Yp_initial))
data_size = len(Data)
r = np.random.permutation(data_size)
Data = Data[r,:]
    
Xp_initial = Data[:110000,start:border]
Yp_initial = Data[:110000,border::step]
    
Xp_mean = np.mean(Xp_initial, axis = 0)
Xp_std = np.std(Xp_initial, axis = 0)
Xp = preprocessing.scale(Xp_initial)

Yp_mean = np.mean(Yp_initial, axis = 0)
Yp_std = np.std(Yp_initial, axis = 0)
Yp = preprocessing.scale(Yp_initial)
# Yp = Yp_initial

Xp_test = Data[110000:,start:border]
Xp_test = (Xp_test - Xp_mean)/Xp_std

Yp_test = Data[110000:,border::step]
Yp_test = (Yp_test - Yp_mean)/Yp_std

In [8]:
fname = './OutputModel/accepted_m_1.pt'
md    = load_checkpoint(fname,device)

NameError: name 'Model' is not defined

In [ ]:
plt.figure(figsize=(6, 5))

plt.plot(range(len(md.total_train_loss)), md.total_train_loss, linewidth=2, label = 'Training loss')
plt.plot(range(len(md.total_train_loss)), md.total_val_loss, linewidth=2, label = 'Validation loss')
axes = plt.gca()
plt.gca().legend(loc=0, prop={'size': 15})
axes.set_ylim([0, 0.1])
axes.set_xlim([0,150])
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
plt.xlabel('Epochs No.', fontsize = 15, labelpad = 10)
plt.ylabel('Loss', fontsize = 15, labelpad = 10)
major_ticks = np.arange(0, len(md.total_train_loss), 20)
minor_ticks = np.arange(0, len(md.total_train_loss), 5)

axes.set_xticks(major_ticks)
axes.set_xticks(minor_ticks, minor=True)

axes.grid(which='both')
axes.grid(color='k', which='minor', alpha=0.2)
axes.grid(color='k', which='major', alpha=0.4)
# axes.set_aspect('equal')
plt.show()

In [ ]:
idx_start = 200
idx_end = 1200
idx_len = idx_end - idx_start

test_1_x = Xp_test[idx_start:idx_end,:]
# test_1_x[110:150] = [None]*(150-110)
test_1_x = test_1_x.reshape(idx_len,299)
test_1_y = Yp_test[idx_start:idx_end,:]
test_1_y = test_1_y.reshape(idx_len,Yp_test.shape[1])
test_tensor_x = torch.tensor(test_1_x).float() # added .values when no scaling in preprocess
test_tensor_y = torch.tensor(test_1_y).float() # added .values when no scaling in preprocess


bs = Yp_test.shape[1]

test_loaded = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y)

test_loader = torch.utils.data.DataLoader(test_loaded,batch_size=bs, shuffle=False)

with torch.no_grad():
    for batch_size_test, (data, target) in enumerate(test_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = md.network(data)
        break
        

prediction = output.numpy()*Yp_std + Yp_mean
targettt = target.numpy()*Yp_std + Yp_mean

plt.figure(figsize=(7, 5))
plt.plot(np.array([0, 4000]), np.array([0, 4000]), 'k--',  linewidth=1)
plt.plot(prediction, targettt, 'o', linewidth=2)

axes = plt.gca()
axes.set_ylim([0, 4000])
axes.set_xlim([0,4000])
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
plt.xlabel('Target values', fontsize = 15, labelpad = 10)
plt.ylabel('Predicted values', fontsize = 15, labelpad = 10)
# plt.grid(color='k', which = 'both', linestyle='--', linewidth=0.5)
major_ticks = np.arange(0,4000, 400)
minor_ticks = np.arange(0,4000, 200)
r2_val = r2_score(prediction[0,:],targettt[0,:])
plt.text(500,2500,r'$r^2:$'+ '{:.4f}'.format(r2_val), fontsize=15)
axes.set_xticks(major_ticks)
axes.set_xticks(minor_ticks, minor=True)

axes.grid(which='both')
axes.grid(color='k', which='minor', alpha=0.2)
axes.grid(color='k', which='major', alpha=0.4)
plt.show()

In [ ]:
idx = 100

test_1_x = Xp_test[idx,:]
test_1_x = test_1_x.reshape(1,299)
test_1_y = Yp_test[idx,:]
test_1_y = test_1_y.reshape(1,Yp_test.shape[1])
test_tensor_x = torch.tensor(test_1_x).float() # added .values when no scaling in preprocess
test_tensor_y = torch.tensor(test_1_y).float() # added .values when no scaling in preprocess


bs = Yp_test.shape[1]

test_loaded = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y)

test_loader = torch.utils.data.DataLoader(test_loaded,batch_size=bs, shuffle=False)

with torch.no_grad():
    for batch_size_test, (data, target) in enumerate(test_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = md.network(data)
        break
        

prediction = output.numpy()*Yp_std + Yp_mean
targettt = target.numpy()*Yp_std + Yp_mean

depth = -np.arange(0,500,4)

plt.figure(figsize=(7, 5))

plt.step(prediction.reshape(Yp_test.shape[1],), depth, linewidth=2, label = 'Prediction')
plt.step(targettt.reshape(Yp_test.shape[1],), depth, linewidth=2, label = 'Target')
axes = plt.gca()
plt.gca().legend(loc=0, prop={'size': 15})
axes.set_ylim([-400, 0])
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
plt.xlabel('Shear Wave Velocity (m/s)', fontsize = 15, labelpad = 10)
plt.ylabel('Depth', fontsize = 15, labelpad = 10)
r2_val = r2_score(prediction[0,:],targettt[0,:])
plt.text(200,-300,r'$r^2:$'+ '{:.4f}'.format(r2_val), fontsize=15)
plt.show()

In [ ]:
idx = 700

test_1_x = Xp_test[idx,:]
# test_1_x[110:150] = [None]*(150-110)
test_1_x = test_1_x.reshape(1,299)
test_1_y = Yp_test[idx,:]
test_1_y = test_1_y.reshape(1,Yp_test.shape[1])
test_tensor_x = torch.tensor(test_1_x).float() # added .values when no scaling in preprocess
test_tensor_y = torch.tensor(test_1_y).float() # added .values when no scaling in preprocess


bs = Yp_test.shape[1]

test_loaded = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y)

test_loader = torch.utils.data.DataLoader(test_loaded,batch_size=bs, shuffle=False)

with torch.no_grad():
    for batch_size_test, (data, target) in enumerate(test_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = md.network(data)
        break
        

prediction = output.numpy()*Yp_std + Yp_mean
targettt = target.numpy()*Yp_std + Yp_mean

depth = -np.arange(0,500,4)

plt.figure(figsize=(7, 5))

plt.step(prediction.reshape(Yp_test.shape[1],), depth, linewidth=2, label = 'Prediction')
plt.step(targettt.reshape(Yp_test.shape[1],), depth, linewidth=2, label = 'Target')
axes = plt.gca()
plt.gca().legend(loc=0, prop={'size': 15})
axes.set_ylim([-400, 0])
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
plt.xlabel('Shear Wave Velocity (m/s)', fontsize = 15, labelpad = 10)
plt.ylabel('Depth', fontsize = 15, labelpad = 10)
r2_val = r2_score(prediction[0,:],targettt[0,:])
plt.text(200,-300,r'$r^2:$'+ '{:.4f}'.format(r2_val), fontsize=15)
plt.show()

In [ ]:
idx = 500

test_1_x = Xp_test[idx,:]
test_1_x = test_1_x.reshape(1,299)
test_1_y = Yp_test[idx,:]
test_1_y = test_1_y.reshape(1,Yp_test.shape[1])
test_tensor_x = torch.tensor(test_1_x).float() # added .values when no scaling in preprocess
test_tensor_y = torch.tensor(test_1_y).float() # added .values when no scaling in preprocess


bs = Yp_test.shape[1]

test_loaded = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y)

test_loader = torch.utils.data.DataLoader(test_loaded,batch_size=bs, shuffle=False)

with torch.no_grad():
    for batch_size_test, (data, target) in enumerate(test_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = md.network(data)
        break
        

prediction = output.numpy()*Yp_std + Yp_mean
targettt = target.numpy()*Yp_std + Yp_mean

# prediction = output.numpy()
# targettt = target.numpy()

depth = -np.arange(0,500,4)

plt.figure(figsize=(7, 5))

plt.step(prediction.reshape(Yp_test.shape[1],), depth, linewidth=2, label = 'Prediction')
plt.step(targettt.reshape(Yp_test.shape[1],), depth, linewidth=2, label = 'Target')
axes = plt.gca()
plt.gca().legend(loc=0, prop={'size': 15})
axes.set_ylim([-400, 0])
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
plt.xlabel('Shear Wave Velocity (m/s)', fontsize = 15, labelpad = 10)
plt.ylabel('Depth', fontsize = 15, labelpad = 10)
r2_val = r2_score(prediction[0,:],targettt[0,:])
plt.text(200,-300,r'$r^2:$'+ '{:.4f}'.format(r2_val), fontsize=15)
plt.show()